Working through [this tutorial](https://graphneural.network/getting-started/)

# Getting Dataset


In [2]:
from spektral.datasets import TUDataset

2022-10-29 11:48:59.248463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dataset = TUDataset('PROTEINS')

100%|█████████████████████████████████████████| 447k/447k [00:00<00:00, 591kB/s]


Successfully loaded PROTEINS.


In [4]:
dataset

TUDataset(n_graphs=1113)

In [8]:
dataset[0]
dataset.filter(lambda g: g.n_nodes < 500)

# Transforming Graphs 

In [9]:
# compute the maximum degree of the dataset, so that we know the size of the one-hot vectors
max_degree = dataset.map(lambda g: g.a.sum(-1).max(), reduce=max)
max_degree

12.0

In [11]:
# augment our node features with the one-hot-encoded degree
from spektral.transforms import Degree


In [13]:
dataset.apply(Degree(int(max_degree)))

In [14]:
dataset[0]

Graph(n_nodes=42, n_node_features=17, n_edge_features=None, n_labels=2)

In [17]:
# following this paper: https://arxiv.org/abs/1609.02907
# so do extra pre-processing of the adjacency matrix

from spektral.transforms import GCNFilter
dataset.apply(GCNFilter())

# every convolutional layer in Spektral has a preprocess(a) method that you can use to transform the adjacency matrix as needed.


# Creating a GNN

In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [20]:
class MyFirstGNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'softmax')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

In [21]:
model = MyFirstGNN(32, dataset.n_labels)
model.compile('adam', 'categorical_crossentropy')

2022-10-29 12:00:59.122595: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
import numpy as np 
np.random.shuffle(dataset)
split = int(0.8 * len(dataset))
dataset_train, dataset_test = dataset[:split], dataset[split:]

/var/folders/86/7p6zt3rd1jz7gwd3wmbkzcw80000gn/T/ipykernel_5376/3333434835.py:2: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(dataset)


In [27]:
#terating over a dataset in mini-batches is not trivial and we cannot simply use the model.fit() bc we can't resize graphs 
# user loader!!

from spektral.data import BatchLoader
loader = BatchLoader(dataset_train, batch_size=32)
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)


/opt/anaconda3/lib/python3.8/site-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Epoch 1/10
28/28 [==============================] - 2s 13ms/step - loss: 7.5035
Epoch 2/10
28/28 [==============================] - 0s 11ms/step - loss: 6.5851
Epoch 3/10
28/28 [==============================] - 0s 11ms/step - loss: 6.0564
Epoch 4/10
28/28 [==============================] - 0s 10ms/step - loss: 5.6047
Epoch 5/10
28/28 [==============================] - 0s 11ms/step - loss: 4.8682
Epoch 6/10
28/28 [==============================] - 0s 10ms/step - loss: 5.5860
Epoch 7/10
28/28 [==============================] - 0s 11ms/step - loss: 5.9103
Epoch 8/10
28/28 [==============================] - 0s 11ms/step - loss: 5.4632
Epoch 9/10
28/28 [==============================] - 0s 10ms/step - loss: 4.1150
Epoch 10/10
28/28 [==============================] - 0s 10ms/step - loss: 3.6491


# Evaluation


In [28]:
loader = BatchLoader(dataset_test, batch_size=32)

In [29]:
loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))

7/7 [==============================] - 0s 13ms/step - loss: 1.1552
Test loss: 1.1552215814590454
